In [3]:
import calendar
import hashlib
import time
import qrcode
import sched
from PIL import Image

class Participant:
    def __init__(self, name, security_deposit):
        self.name = name
        self.security_deposit = int(security_deposit)
        self.is_registered = False
        self.is_client = False
        self.is_manufacturer = False
        self.is_distributor = False
        self.is_distributing = False

class Transactions:
    def __init__(self, client_id, distributor_id, product_id, trans_id):
        self.ID = trans_id
        self.client_id = client_id
        self.distributor_id = distributor_id
        self.product_id = product_id
        self.transaction_id = trans_id
        self.timestamp_1 = None  # as per mentioned in document.
        self.timestamp_2 = None
        self.timestamp_3 = None
        self.time_Start = calendar.timegm(time.gmtime())

    def set_timestamp_1(self):
        self.timestamp_1 = time.time()

    def set_timestamp_2(self):
        self.timestamp_2 = time.time()

    def set_timestamp_3(self):
        self.timestamp_3 = time.time()


class Block:
    def __init__(self, index, previous_hash, timestamp, transactions, nonce=0):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.transactions = transactions
        self.nonce = nonce
        self.merkle_root = self.calculate_merkle_root()
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        data_str = str(self.index) + self.previous_hash + str(self.timestamp) + str(self.merkle_root) + str(self.nonce)
        return hashlib.sha256(data_str.encode()).hexdigest()

    def calculate_merkle_root(self):
        if not self.transactions:
            return "0"

        merkle_tree = [hashlib.sha256(tx.encode()).hexdigest() for tx in self.transactions]
        while len(merkle_tree) > 1:
            if len(merkle_tree) % 2 != 0:
                merkle_tree.append(merkle_tree[-1])
            merkle_tree = [hashlib.sha256((merkle_tree[i] + merkle_tree[i + 1]).encode()).hexdigest()
                           for i in range(0, len(merkle_tree), 2)]

        return merkle_tree[0]


class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.participants = {}  # Dictionary to store participants (clients, distributors, manufacturer)
        self.difficulty = 4  # PoW difficulty level
        self.product_status = {}  # Dictionary to track product status
        self.issue_log = []  # List to track distribution and receipt issues
        self.count_manufacturer = 0
        self.t_id = 0
        self.transactions = {}
        self.pending_transactions = []
        self.completed_transactions = []
        self.list_transactions = []

    def create_genesis_block(self):
        return Block(0, "0", int(time.time()), ["Genesis Transaction"])

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.timestamp = int(time.time())
        new_block.hash = self.proof_of_work(new_block)
        self.chain.append(new_block)

    def proof_of_work(self, block):
        prefix = '0' * self.difficulty

        while block.hash[:self.difficulty] != prefix:
            block.nonce += 1
            block.hash = block.calculate_hash()

        return block.hash

    def mine_block(self, transactions):
        new_block = Block(len(self.chain), self.get_latest_block().hash, int(time.time()), transactions)
        new_block.hash = self.proof_of_work(new_block)
        self.chain.append(new_block)

    def register_participant(self, address, name, security_deposit, option):
        if address not in self.participants:
            self.participants[address] = Participant(name, security_deposit)
            self.participants[address].is_registered = True
            if option == 'Client':
                self.participants[address].is_client = True
            if option == 'Manufacturer':
                self.participants[address].is_manufacturer = True
            if option == 'Distributor':
                self.participants[address].is_distributor = True
            print("Registered Particpant")
        else:
            print("This ID isnt unique")

        #print(self.participants)

    def distribute_product(self, distributor_address, client_address, product_id):
        if distributor_address not in self.participants or client_address not in self.participants:
            # result_label.config(text= "Invalid distributor or client")
            return "Invalid distributor or client"
        if not self.participants[distributor_address].is_distributor:
            # result_label.config(text= "Invalid distributor")
            return "Invalid distributor"
        if not self.participants[client_address].is_client:
            # result_label.config(text= "Invalid client.")
            return "Invalid client."
        if self.participants[distributor_address].is_distributing:
            # result_label.config(text= "Distributer Busy")
            return "Distributer Busy"

        distributor = self.participants[distributor_address]
        self.participants[distributor_address].is_distributing = True
        client = self.participants[client_address]

        if distributor.security_deposit < 1:
            # result_label.config(text= "Distributor doesn't have enough security deposit.")
            return "Distributor doesn't have enough security deposit."

        if client.security_deposit < 1:
            # result_label.config(text= "Client doesn't have enough security deposit.")
            return "Client doesn't have enough security deposit."

        if product_id in self.product_status:
            # result_label.config(text= "Product has already been distributed.")
            return "Product has already been distributed."

        self.product_status[product_id] = [distributor_address, client_address]
        distributor.security_deposit -= 1
        client.security_deposit -= 1

        transaction = Transactions(client_address, distributor_address, product_id, self.t_id)
        self.transactions[self.t_id] = transaction
        self.list_transactions.append(self.t_id)
        self.pending_transactions.append(self.t_id)
        self.t_id += 1

        print(self.transactions[transaction.transaction_id].transaction_id)
        return f"Order recorded with transaction ID: {transaction.transaction_id}"

    def received_from_manufacturer(self, distributor_id, transaction_id):
        print(self.transactions, transaction_id)
        if transaction_id not in self.transactions:
            print("Transaction ID is invalid")
            return
        if self.transactions[transaction_id].distributor_id != distributor_id:
            print("Invalid distributor")
            return
        self.transactions[transaction_id].set_timestamp_1()
        print(f"Product {self.transactions[transaction_id].product_id} received from manufacturer.")

    def dispatched_product(self, distributor_id, transaction_id):
        if transaction_id not in self.transactions:
            print("Transaction ID is invalid")
            return
        if self.transactions[transaction_id].distributor_id != distributor_id:
            print("Invalid distributor")
            return
            # Check if the distributor has received the product from the manufacturer
        if self.transactions[transaction_id].timestamp_1 is None:
            print("Distributor hasn't received the product from the manufacturer yet.")
            return
        self.transactions[transaction_id].set_timestamp_2()
        print(f"Product {self.transactions[transaction_id].product_id} dispatched.")

    def client_received_product(self, client_id, transaction_id):
        if transaction_id not in self.transactions:
            print("Transaction ID is invalid")
            return
        if self.transactions[transaction_id].client_id != client_id:
            print("Invalid client")
            return
        if self.transactions[transaction_id].timestamp_1 is None:
            print("Distributor hasn't received the product from the manufacturer yet.")
            return
            # Check if the product has been dispatched by the distributor
        if self.transactions[transaction_id].timestamp_2 is None:
             print(f"Product {self.transactions[transaction_id].product_id} hasn't been dispatched by the distributor")
             return
        self.transactions[transaction_id].set_timestamp_3()
        print(f"Product {self.transactions[transaction_id].product_id} has been received by client.")
        distributor_id = self.transactions[transaction_id].distributor_id
        self.participants[distributor_id].is_distributing = False


    def generate_qr_code(self,transaction_id):
        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,)

        if transaction_id not in self.list_transactions:
          print("This transaction ID doesnt exist")
          return

        if transaction_id not in self.pending_transactions:
          block_info = " Current status of transaction is pending "
          qr.add_data(block_info)
          qr.make(fit=True)
          img = qr.make_image(fill_color="black", back_color="white")
          img.save("code.png")

        else:
          block_info = " Current status of transaction is Completed "
          qr.add_data(block_info)
          qr.make(fit=True)
          img = qr.make_image(fill_color="black", back_color="white")
          img.save("code.png")


def check_pending_transactions(scheduler):
    for i in blockchain.pending_transactions:
      transaction = id_trans[i]
      if transaction.Timestamp_1 and transaction.Timestamp_2 and transaction.Timestamp_3:
        blockchain.pending_transactions.remove(i)
        blockchain.completed_transactions.append(i)
        blockchain.mine_block(transaction)

if __name__ == '__main__':
    blockchain = Blockchain()

    my_scheduler = sched.scheduler(time.time, time.sleep)
    my_scheduler.enter(60, 1, check_pending_transactions, (my_scheduler,))
    my_scheduler.run()

    # Registering participants through the input terminal
    while True:
        print("Select an option:")
        print("1. Register Manufacturer")
        print("2. Register Distributor")
        print("3. Register Client")
        print("4. Client requests for product")
        print("5. Distributor receives the product from the manufacturer")
        print("6. Distributor dispatches the product")
        print("7. Client receives the product")
        print("8. Generate QR Code")
        print("9. Exit")

        option = input("Enter your choice: ")

        if option == "1":
            manufacturer_address = input("Enter Manufacturer's ID: ")
            manufacturer_name = input("Enter Manufacturer's Name: ")
            manufacturer_security_deposit = int(input("Enter Manufacturer's Security Deposit: "))
            blockchain.register_participant(manufacturer_address, manufacturer_name, manufacturer_security_deposit,
                                            'Manufacturer')
            print("Manufacturer registered.")

        elif option == "2":
            distributor_address = input("Enter Distributor's ID: ")
            distributor_name = input("Enter Distributor's Name: ")
            distributor_security_deposit = int(input("Enter Distributor's Security Deposit: "))
            blockchain.register_participant(distributor_address, distributor_name, distributor_security_deposit,
                                            'Distributor')
            print("Distributor registered.")

        elif option == "3":
            client_address = input("Enter Client's ID: ")
            client_name = input("Enter Client's Name: ")
            client_security_deposit = int(input("Enter Client's Security Deposit: "))
            blockchain.register_participant(client_address, client_name, client_security_deposit, 'Client')
            print("Client registered.")

        elif option == "4":
            client_address = input("Enter Client's ID: ")
            distributor_address = input("Enter Distributor's ID: ")
            while blockchain.participants[distributor_address].is_distributing:
                distributor_address = input("Enter another Distributor's ID: ")
            product_id = input("Enter Product ID: ")
            result = blockchain.distribute_product(distributor_address, client_address, product_id)

            print(result)

        elif option == "5":
            distributor_address = input("Enter Distributor's ID: ")
            transaction_id = int(input("Enter Transaction ID: "))
            blockchain.received_from_manufacturer(distributor_address, transaction_id)

        elif option == "6":
            distributor_address = input("Enter Distributor's ID: ")
            transaction_id = int(input("Enter Transaction ID: "))
            blockchain.dispatched_product(distributor_address, transaction_id)

        elif option == "7":
            client_address = input("Enter Client's ID: ")
            transaction_id = int(input("Enter Transaction ID: "))
            blockchain.client_received_product(client_address, transaction_id)

        elif option == "8":
          print("Enter transaction_ID")
          x = int(input())
          blockchain.generate_qr_code(x)
          print("QR Code generated as code.png")

        elif option == "9":
            break

        else:
            print("Plase enter a valid option")
            continue

    # Print registered participants
    print("\nRegistered Participants:")
    for address, participant in blockchain.participants.items():
        print(f"Participant Address: {address}")
        print(f"Name: {participant.name}")
        print(f"Security Deposit: {participant.security_deposit}")
        print(f"Registered: {participant.is_registered}\n")


Select an option:
1. Register Manufacturer
2. Register Distributor
3. Register Client
4. Client requests for product
5. Distributor receives the product from the manufacturer
6. Distributor dispatches the product
7. Client receives the product
8. Generate QR Code
9. Exit
Registered Particpant
Manufacturer registered.
Select an option:
1. Register Manufacturer
2. Register Distributor
3. Register Client
4. Client requests for product
5. Distributor receives the product from the manufacturer
6. Distributor dispatches the product
7. Client receives the product
8. Generate QR Code
9. Exit
Registered Particpant
Distributor registered.
Select an option:
1. Register Manufacturer
2. Register Distributor
3. Register Client
4. Client requests for product
5. Distributor receives the product from the manufacturer
6. Distributor dispatches the product
7. Client receives the product
8. Generate QR Code
9. Exit
Registered Particpant
Client registered.
Select an option:
1. Register Manufacturer
2. Regi